# Валидация системы вопросов и ответов

## Описание проекта

Данный проект представляет собой систему вопросов и ответов, состоящую из трех микросервисов:

1. **Search Service** (порт 8001)
   - Использует модель SentenceTransformer для создания эмбеддингов
   - Осуществляет семантический поиск по базе знаний
   - Возвращает наиболее релевантные фрагменты текста

2. **LLM Service** (порт 8002)
   - Использует Mistral-7B для генерации ответов
   - Принимает контекст из Search Service
   - Генерирует структурированные ответы

3. **Router Service** (порт 8003)
   - Координирует взаимодействие между сервисами
   - Обрабатывает входящие запросы
   - Возвращает финальные ответы с метаданными

## Архитектура решения

```
Client -> Router Service -> Search Service -> База знаний
                        -> LLM Service    -> Генерация ответа
```


In [8]:
import requests
import json
from typing import List, Dict, Any
from pydantic import BaseModel
import logging
from datetime import datetime
import time

# Конфигурация
ROUTER_URL = "http://127.0.0.1:8003/ask"
SEARCH_URL = "http://127.0.0.1:8001/search"
LLM_URL = "http://127.0.0.1:8002/generate"

# Константы
MAX_TOP_K = 5
QUESTIONS = [
    'Where can I apply Convolutional Neural Network?',
    'What is Reinforcement Learning?',
    'How to deploy a machine learning model?',
    'How to implement a random forest algorithm?'
]

def send_question(question: str) -> Dict[str, Any]:
    """Отправка вопроса и получение ответа"""
    try:
        response = requests.post(
            ROUTER_URL,
            json={"question": question},
            timeout=360
        )
        response.raise_for_status()
        return response.json()
    except Exception as e:
        print(f"Error processing question: {str(e)}")
        return None

In [10]:


# Настройка логирования
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s'
)
logger = logging.getLogger(__name__)


def search_documents(query: str, top_k: int = 3) -> List[Dict]:
    """
    Поиск релевантных документов
    
    Args:
        query (str): Поисковый запрос
        top_k (int): Количество результатов (не более 5)
    
    Returns:
        List[Dict]: Список найденных документов
    """
    # Ограничиваем top_k
    top_k = min(max(1, top_k), MAX_TOP_K)
    
    payload = {
        "query": query,
        "top_k": top_k
    }
    
    try:
        logger.info(f"Sending search request: {json.dumps(payload, indent=2)}")
        response = requests.post(SEARCH_URL, json=payload)
        response.raise_for_status()
        
        results = response.json()
        logger.info(f"Found {len(results)} documents")
        
        # Вывод результатов
        print("\nSearch Results:")
        print("="*50)
        for i, doc in enumerate(results, 1):
            print(f"\nDocument {i}:")
            print(f"Title: {doc['title']}")
            print(f"Score: {doc['score']:.4f}")
            print(f"Content: {doc['content'][:200]}...")
        
        # Вывод метрик
        print("\nSearch Metrics:")
        print("="*50)
        print(f"Response time: {response.elapsed.total_seconds():.2f} seconds")
        print(f"Results found: {len(results)}")
        print(f"Average relevance score: {sum(doc['score'] for doc in results)/len(results):.4f}")
        
        return results
        
    except Exception as e:
        logger.error(f"Search error: {str(e)}")
        raise



**Функция search_documents:**
Функция search_documents:
Ограничивает top_k до 5
Выводит результаты поиска
Показывает метрики поиска
Возвращает результаты для дальнейшего использованияания

In [12]:
def ask_llm(question: str, context: List[Dict], max_context: int = 3) -> Dict:
    """
    Отправка запроса к LLM
    
    Args:
        question (str): Вопрос
        context (List[Dict]): Контекст из поиска
        max_context (int): Максимальное количество документов для контекста
    
    Returns:
        Dict: Ответ LLM
    """
    # Ограничиваем количество документов
    context = context[:min(len(context), max_context)]
    
    payload = {
        "question": question,
        "context": [
            {
                "content": doc["content"],
                "title": doc["title"]
            }
            for doc in context
        ]
    }
    
    try:
        logger.info(f"Sending LLM request with {len(context)} documents")
        response = requests.post(LLM_URL, json=payload)
        response.raise_for_status()
        
        result = response.json()
        
        # Вывод результата
        print("\nLLM Response:")
        print("="*50)
        print(f"Answer: {result['answer']}")
        print(f"\nMetrics:")
        print(f"Model: {result['llm_name']}")
        print(f"Latency: {result['latency']:.2f} seconds")
        
        return result
        
    except Exception as e:
        logger.error(f"LLM error: {str(e)}")
        raise



In [ ]:
**Функция ask_llm:**

Принимает результаты поиска
Форматирует контекст для LLM
Отправляет запрос
Выводит ответ и метрики
Возвращает полный ответ

In [19]:
# Поиск ответа на вопрос 1 Where can I apply Convolutional Neural Network?:
if __name__ == "__main__":
    question = QUESTIONS[0]
    
    # Поиск документов
    search_results = search_documents(question, top_k=3)
    
    # Запрос к LLM
    llm_response = ask_llm(question, search_results, max_context=2)

2024-11-05 13:54:00,433 - INFO - Sending search request: {
  "query": "Where can I apply Convolutional Neural Network?",
  "top_k": 3
}
2024-11-05 13:54:00,471 - INFO - Found 3 documents
2024-11-05 13:54:00,472 - INFO - Sending LLM request with 2 documents



Search Results:

Document 1:
Title: Convolutional Neural Network: A Step By Step Guide
Score: 0.6174
Content: Convolutional Neural Network: A Step By Step Guide Shashikant · Follow Published in Towards Data Science · 9 min read · Mar 17, 2019 -- 1 Listen Share

“Artificial Intelligence, deep learning, machine...

Document 2:
Title: An introduction to Convolutional Neural Networks
Score: 0.5768
Content: An introduction to Convolutional Neural Networks

A Convolutional neural network (CNN) is a neural network that has one or more convolutional layers and are used mainly for image processing, classific...

Document 3:
Title: Connectivity Patterns in Deep Neural Networks
Score: 0.5729
Content: This article will discuss a central component that is driving progress in Neural Network design, namely in Convolutional Networks for Computer Vision tasks. Classical CNNs such as LeNet-5, AlexNet, an...

Search Metrics:
Response time: 0.04 seconds
Results found: 3
Average relevance score: 0.5890

L

In [23]:
# Поиск ответа на вопрос 2 What is Reinforcement Learning?:
if __name__ == "__main__":
    question = QUESTIONS[1]
    
    # Поиск документов
    search_results = search_documents(question, top_k=3)
    
    # Запрос к LLM
    llm_response = ask_llm(question, search_results, max_context=2)

2024-11-05 13:56:45,452 - INFO - Sending search request: {
  "query": "What is Reinforcement Learning?",
  "top_k": 3
}
2024-11-05 13:56:45,488 - INFO - Found 3 documents
2024-11-05 13:56:45,490 - INFO - Sending LLM request with 2 documents



Search Results:

Document 1:
Title: Reinforcement Learning : Markov-Decision Process (Part 1)
Score: 0.7003
Content:  Reinforcement Learning is all about goal to maximize the reward.So, let’s add reward to our Markov Chain.This gives us Markov Reward Process.

Markov Reward Process : As the name suggests, MDPs are t...

Document 2:
Title: Simple Reinforcement Learning: Q-learning
Score: 0.6977
Content: Typical Exploring Image for RL - Credit @mike.shots

Introduction

One of my favorite algorithms that I learned while taking a reinforcement learning course was q-learning. Probably because it was the...

Document 3:
Title: Teaching A Computer To Land On The Moon
Score: 0.6802
Content: ments. The program doing the learning and control is referred to as an agent.

Agents that learn the correct approach to solving problems without being presented with lots of solved examples are doing...

Search Metrics:
Response time: 0.03 seconds
Results found: 3
Average relevance score: 0.6927

LLM Res

In [25]:
# Поиск ответа на вопрос 3 How to deploy a machine learning model?:
if __name__ == "__main__":
    question = QUESTIONS[2]
    
    # Поиск документов
    search_results = search_documents(question, top_k=3)
    
    # Запрос к LLM
    llm_response = ask_llm(question, search_results, max_context=2)

2024-11-05 13:58:04,054 - INFO - Sending search request: {
  "query": "How to deploy a machine learning model?",
  "top_k": 3
}
2024-11-05 13:58:04,091 - INFO - Found 3 documents
2024-11-05 13:58:04,091 - INFO - Sending LLM request with 2 documents



Search Results:

Document 1:
Title: ML Models — Prototype to Production
Score: 0.7547
Content: significant challenge. As a data scientist and an ML practitioner, I have myself experienced that it is often more difficult to make the journey from a reliable and accurate prototype model to a well-...

Document 2:
Title: ML Models — Prototype to Production
Score: 0.6834
Content: of creating production machine learning services.

The deployment and operational aspects of “productionizing” ML models lie at the intersection of various practices and disciplines like Statistical m...

Document 3:
Title: How Microsoft Azure Machine Learning Studio Clarifies Data Science
Score: 0.6128
Content: , Studio even makes deployment of models as a web service easy.

But as I said in the title, Studio is not a panacea that allows anyone to build machine learning models. Machine learning is complex an...

Search Metrics:
Response time: 0.04 seconds
Results found: 3
Average relevance score: 0.6836

LLM Resp

In [29]:
# Поиск ответа на вопрос 4 How to implement a random forest algorithm?:
if __name__ == "__main__":
    question = QUESTIONS[3]
    
    # Поиск документов
    search_results = search_documents(question, top_k=3)
    
    # Запрос к LLM
    llm_response = ask_llm(question, search_results, max_context=2)

2024-11-05 15:04:46,606 - INFO - Sending search request: {
  "query": "How to implement a random forest algorithm?",
  "top_k": 3
}
2024-11-05 15:04:46,645 - INFO - Found 3 documents
2024-11-05 15:04:46,646 - INFO - Sending LLM request with 2 documents



Search Results:

Document 1:
Title: Predict Employee Retention
Score: 0.6459
Content: represents a decision rule, and each leaf node represents the outcome. The topmost node in a decision tree is known as the root node. It learns to partition based on the attribute value. It partitions...

Document 2:
Title: Machine Learning Algorithms In Layman’s Terms, Part 2
Score: 0.6449
Content: o build little decision trees, each one of which is built simultaneously with random subsets of your data.

…But There’s More!

Not only does each tree in a Random Forest model only contain a subset o...

Document 3:
Title: Machine Learning Model Comparison for Breast Cancer Classification and Bio-Marker Identification
Score: 0.6336
Content:  forest model in hopes of achieving better accuracy and a more transparent machine model.

This code builds a random search to identify the best parameters for the random forest to make its classifica...

Search Metrics:
Response time: 0.04 seconds
Results found: 3
Av

Обе функции логируются и обрабатывают ошибки, выводят метрики производительности.
Можно использовать их по отдельности или вместе, регулируя количество документов для поиска и контекста для LLM


### Выводы

1. **Качество ответов**:
   Ответы релевантны запросам, но запросы носят очень общий характер и ответы подбираются соответственно. Можно улучшить выдачу, используя переформулировку вопросов.
   Время обработки запросов - 0.03 - 0.04 секунды. Можно использовать Redis для скорости подгрузки информации.
   В проекте в качестве метрики использовалась similarity, можно использовать дополнительные метрики, например, bm25. Также я хотела, но не успела попробовать другой вариант выбачи результатов (bi-encoder + cross-encoder).
    Также можно расширенить базу знаний, чтобы иметь более профильные ответы.
2. **Итоговый ответ LLM**:
   Можно использовать в чат-ботах. В нашем валидационном ноутбуке видно, что ответ релевантен и базируется на полученном контексте из базы знаний, но расширен самой LLM. Это можно исправить, снижая температуру (сейчас 0.5) и подавая один самый релевантный ответ на вход в LLM. Ответ LLM пока выполняется долго, минимальное время - 22 секунды, на текущих вопросах - 26- 46 секунд.
3. **Использование сервиса**
   Сервис имеет документацию, папка DOCS с описанием.
   
   